Реализуйте максимально эффективным образом алгоритм иерархической кластеризации с алгоритмом объединения single-link clustering для точек, расположенных на прямой. Опишите алгоритм, а также найдите пример, когда подобный алгоритм показывает «плохие» результаты.

Описание алгоритма:

Пусть дан массив _points_ точек в n-мерном пространстве, лежащих на одной прямой. 

Массив кластеров _clusters_ = $[0, 1, .. , k]$, в каждом кластере одна точка. Этот массив до конца алгоритма не меняется.

Зафиксируем одну из координат пространства и отсортируем исходные точки по значению этой координаты. Например точки $(2, 0, -3), (-2, 6, 9), (0, 3, 4)$ - отсортированные по первой координате, будет $(-2, 6, 9), (0, 3, 4), (2, 0, 3)$. Получим массив точек $[p_0, p_1, .. , p_k]$. Пройдем по этому массиву и найдем расстояния между соседними точками _dists_ = $[d(p_0, p_1), d(p_1, p_2), .. , d(p_{k-1}, p_k)]$.

Далее, на каждой итерации выбираем минимальный элемент из dists. Элемент на позиции _j_ соответствует расстоянию между точками $p_j$ и $p_{j+1}$. Сливаем кластеры, содержащие эти точки.

Используется несколько вспомогательных массивов. 

_evolution_ - массив очередей. Когда мы сливаем сливаем кластеры двух точек $i$ и $j$ $(i < k, j < k)$ в новый кластер $s$ мы добавляем $s$ в очереди _evolution[i]_ и _evolution[j]_. 

_history_ - когда сливаем кластеры точек $i$ и $j$, добавляем в _history_ кортеж $(i, j, d(i, j))$.

Чтобы восстановить дендрограмму:

идем по массиву _history_. Когда сливаются $i$ и $j$, достаем элементы _evolution[i]_ и _evolution[j]_ - на момент этого слияния точки, которые изначально были в $i$ и $j$ кластере, находятся в кластерах _evolution[i]_ и _evolution[j]_.


сложность:

* сортировка исходных точек - O(nlogn)
* посчитать _dists_ - O(n)
* три цикла - O(n)
* итого временная сложность O(nlogn)

дополнительная память:
* массивы $dists, clusters, evolution, history$ - все O(n)
* итого O(n) дополнительной памяти.


навеено идеей SLINK

In [13]:
import numpy as np
from queue import Queue

#dissimilarity between points
def dc(p1, p2):
    return np.linalg.norm(p1-p2)

#distances between adjacent points [points - sorted list]
def point_dists(points):
    return [dc(points[i], points[i+1]) for i in range(len(points)-1)]

#linkage matrix
def linkage(points):
    # print(points)
    linkage_matrix = []
    n = len(points)
    
    points_argsort = np.argsort(points[:, 0])
    points = points[points_argsort]
    # print(points)
    dists = point_dists(points)
    dists_argsort = np.argsort(dists)

    clusters = np.array(range(n))[points_argsort]
    cluster_sizes = [1 for i in range(n)]
    evolution = []
    for i in range(n):
        evolution.append(Queue())
        evolution[i].put(i)

    history = []

    for i, min_id in enumerate(dists_argsort):
        cl1, cl2 = clusters[min_id], clusters[min_id + 1]
        evolution[cl1].put(n+i)
        evolution[cl2].put(n+i)
        history.append((cl1, cl2, dists[min_id]))

    for i, h in enumerate(history):
        cl1, cl2, d = evolution[h[0]].get(), evolution[h[1]].get(), h[2]
        cluster_sizes.append(cluster_sizes[cl1] + cluster_sizes[cl2])
        linkage_matrix.append([cl1, cl2, d, cluster_sizes[-1]])

    return linkage_matrix

In [14]:
points = np.array([[3*i, i, i] for i in [2, 8, 0, 4, 1, 9, 9, 0]])
print(np.array(linkage(points)))

[[ 2.          7.          0.          2.        ]
 [ 5.          6.          0.          2.        ]
 [ 8.          4.          3.31662479  3.        ]
 [10.          0.          3.31662479  4.        ]
 [ 1.          9.          3.31662479  3.        ]
 [11.          3.          6.63324958  5.        ]
 [13.         12.         13.26649916  8.        ]]


In [12]:
from scipy.cluster.hierarchy import linkage as scipy_linkage

Z = scipy_linkage(points, 'single')
print(Z)

[[ 2.          7.          0.          2.        ]
 [ 5.          6.          0.          2.        ]
 [ 0.          4.          3.31662479  2.        ]
 [ 8.         10.          3.31662479  4.        ]
 [ 1.          9.          3.31662479  3.        ]
 [ 3.         11.          6.63324958  5.        ]
 [12.         13.         13.26649916  8.        ]]
